In [1]:
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
from pathlib import Path
import time
import pandas as pd
from scipy.spatial.distance import dice
from utils import plot3, padd_images_to_max, register_image, plot1, print_img_info, resample2target, convert_itk_to_nda, convert_nda_to_itk, remap_labels
from EM import ExpectationMaximization
from tqdm import tqdm
import nibabel as nib
from nibabel.processing import conform

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
pd_val_path = Path().resolve() / 'proc_data/Validation_Set/'
pd_train_path = Path().resolve() / 'proc_data/Training_Set/'
pd_test_path = Path().resolve() / 'proc_data/Test_Set/'
val_path = Path().resolve() / 'data/Validation_Set/'
train_path = Path().resolve() / 'data/Training_Set/'
test_path = Path().resolve() / 'data/Test_Set/'

In [6]:
# generate the merged labels images for different sets
results_synthseg = []
for val_set in pd_test_path.iterdir():
    if val_set.is_dir():
        synthseg_o = sitk.ReadImage(str(val_set/f'{val_set.name}_seg_resampled.nii.gz'))
        synthseg = convert_nda_to_itk(remap_labels(convert_itk_to_nda(synthseg_o),
                                        mapping_dict={0: [0, 24], #background
                                                      1: [4, 43, 14, 15, 44, 5], #CSF
                                                      2: [3, 8, 42, 47, 10, 49, 11, 50, 13, 52, 12, 51, 17, 53, 18, 54, 26, 58, 28, 60], #GM
                                                      3: [2, 16, 7, 41, 46] #WM
                                                      }), synthseg_o)
        sitk.WriteImage(synthseg, str(val_set/f'{val_set.name}_seg_resampled_merged.nii.gz'))

In [14]:
ss_labels = pd.read_csv(pd_val_path.parent/'synthseg_labels.txt', sep=' ', header=0)
ss_labels

,labels,structures
0,0,background
1,2,left_cerebral_white_matter
2,3,left_cerebral_cortex
3,4,left_lateral_ventricle
4,5,left_inferior_lateral_ventricle
5,7,left_cerebellum_white_matter
6,8,left_cerebellum_cortex
7,10,left_thalamus
8,11,left_caudate
9,12,left_putamen


In [58]:
ss_labels.loc[ss_labels['structures'].str.contains('white')]

,labels,structures
1,2,left_cerebral_white_matter
5,7,left_cerebellum_white_matter
18,41,right_cerebral_white_matter
22,46,right_cerebellum_white_matter


In [57]:
mapping_dict = {0: [0], #background
                1: [24, 4, 43, 14, 15, 44, 5], #CSF
                2: [3, 8, 42, 47, 10, 49, 11, 50, 13, 52, 12, 51, 17, 53, 18, 54, 26, 58, 28, 60], #GM
                3: [2, 16, 7, 41, 46] #WM
                }
labels_set = set(ss_labels['labels'].values)
labels_set.difference(set([item for sublist in mapping_dict.values() for item in sublist]))

set()

In [70]:
# merging synthseg labels to GM, WM, CSF
ss_img_itk = sitk.ReadImage(str(pd_val_path/'IBSR_11/IBSR_11_seg_resampled.nii.gz'))
ss_img = convert_itk_to_nda(ss_img_itk)
np.unique(ss_img, return_counts=True)

(array([ 0,  2,  3,  4,  5,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 24,
        26, 28, 41, 42, 43, 44, 46, 47, 49, 50, 51, 52, 53, 54, 58, 60],
       dtype=int32),
 array([7391319,  161132,  156809,    4587,     349,   11122,   35039,
           4161,    2246,    3223,    1131,     606,    1381,   13973,
           2899,    1147,  200227,     394,    2841,  165643,  161120,
           3761,     499,   11300,   33907,    4363,    2205,    3188,
           1159,    2799,    1121,     415,    2542]))

In [71]:
# replace values of ss_img with new labels
ss_img_new = np.zeros(ss_img.shape)
for key, value in mapping_dict.items():
    print(key, value)
    for v in value:
        ss_img_new[ss_img == v] = key
np.unique(ss_img_new, return_counts=True)

0 [0]
1 [24, 4, 43, 14, 15, 44, 5]
2 [3, 8, 42, 47, 10, 49, 11, 50, 13, 52, 12, 51, 17, 53, 18, 54, 26, 58, 28, 60]
3 [2, 16, 7, 41, 46]


(array([0., 1., 2., 3.]), array([7391319,  211410,  422709,  363170]))

In [72]:
sitk.WriteImage(convert_nda_to_itk(ss_img_new, ss_img_itk), str(pd_val_path/'IBSR_11/IBSR_11_seg_resampled_merged.nii.gz'))

### Resample SynthSeg segmentations to original dimensions and voxel spacing

In [20]:
results_resampling = []
for val_set in pd_val_path.iterdir():
    case_results = {}
    if val_set.is_dir():
        # get original spacing and shape
        case_results['case'] = val_set.name
        o_img = nib.load(str(val_path/f'{val_set.name}/{val_set.name}.nii.gz'))
        case_results['o_spacing'] = o_img.header.get_zooms()
        case_results['o_shape'] = o_img.header.get_data_shape()
        
        # read and resample back the segmentation
        seg_img = nib.load(str(val_set/f'{val_set.name}_sseg.nii.gz'))
        case_results['pd_spacing'] = seg_img.header.get_zooms()
        case_results['pd_shape'] = seg_img.header.get_data_shape()
        seg_img_resampled = conform(seg_img, out_shape=case_results['o_shape'][:-1], 
                                                    voxel_size=case_results['o_spacing'][:-1], order=0)
        nib.save(seg_img_resampled, str(val_set/f'{val_set.name}_seg_resampled.nii.gz'))

        # read and resample back the resampled image
        img_rs = nib.load(str(val_set/f'{val_set.name}_resmp.nii.gz'))
        img_rs_back = conform(img_rs, out_shape=case_results['o_shape'][:-1], 
                                                    voxel_size=case_results['o_spacing'][:-1], order=4)
        nib.save(img_rs_back, str(val_set/f'{val_set.name}_resmp_back.nii.gz'))  

        case_results['new_spacing'] = seg_img_resampled.header.get_zooms()
        case_results['new_shape'] = seg_img_resampled.header.get_data_shape()
        results_resampling.append(case_results)
pd.DataFrame(results_resampling)

,case,o_spacing,o_shape,pd_spacing,pd_shape,new_spacing,new_shape
0,IBSR_11,"(1.0, 1.5, 1.0, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(256, 192, 256)","(1.0, 1.5, 1.0)","(256, 128, 256)"
1,IBSR_12,"(1.0, 1.5, 1.0, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(256, 192, 256)","(1.0, 1.5, 1.0)","(256, 128, 256)"
2,IBSR_13,"(0.9375, 1.5, 0.9375, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(240, 192, 240)","(0.9375, 1.5, 0.9375)","(256, 128, 256)"
3,IBSR_17,"(0.8370536, 1.5, 0.8370536, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(215, 192, 215)","(0.8370536, 1.5, 0.8370536)","(256, 128, 256)"
4,IBSR_14,"(0.9375, 1.5, 0.9375, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(240, 192, 240)","(0.9375, 1.5, 0.9375)","(256, 128, 256)"


In [22]:
results = []
for val_set in pd_val_path.iterdir():
    case_results = {}
    if val_set.is_dir():
        case_results['case'] = val_set.name
        seg_img = nib.load(str(val_set/f'{val_set.name}_sseg.nii.gz')).get_fdata()
        seg_img_resamp = nib.load(str(val_set/f'{val_set.name}_seg_resampled.nii.gz')).get_fdata()
        case_results['equal'] = (np.unique(seg_img) == np.unique(seg_img_resamp)).all()
        results.append(case_results)
pd.DataFrame(results)

,case,equal
0,IBSR_11,True
1,IBSR_12,True
2,IBSR_13,True
3,IBSR_17,True
4,IBSR_14,True


In [24]:
results_resampling = []
for val_set in pd_train_path.iterdir():
    case_results = {}
    if val_set.is_dir():
        # get original spacing and shape
        case_results['case'] = val_set.name
        o_img = nib.load(str(train_path/f'{val_set.name}/{val_set.name}.nii.gz'))
        case_results['o_spacing'] = o_img.header.get_zooms()
        case_results['o_shape'] = o_img.header.get_data_shape()
        
        # read and resample back the segmentation
        seg_img = nib.load(str(val_set/f'{val_set.name}_sseg.nii.gz'))
        case_results['pd_spacing'] = seg_img.header.get_zooms()
        case_results['pd_shape'] = seg_img.header.get_data_shape()
        seg_img_resampled = conform(seg_img, out_shape=case_results['o_shape'][:-1], 
                                                    voxel_size=case_results['o_spacing'][:-1], order=0)
        nib.save(seg_img_resampled, str(val_set/f'{val_set.name}_seg_resampled.nii.gz'))

        # read and resample back the resampled image
        img_rs = nib.load(str(val_set/f'{val_set.name}_resmp.nii.gz'))
        img_rs_back = conform(img_rs, out_shape=case_results['o_shape'][:-1], 
                                                    voxel_size=case_results['o_spacing'][:-1], order=4)
        nib.save(img_rs_back, str(val_set/f'{val_set.name}_resmp_back.nii.gz'))  

        case_results['new_spacing'] = seg_img_resampled.header.get_zooms()
        case_results['new_shape'] = seg_img_resampled.header.get_data_shape()
        results_resampling.append(case_results)
pd.DataFrame(results_resampling)

,case,o_spacing,o_shape,pd_spacing,pd_shape,new_spacing,new_shape
0,IBSR_18,"(0.8370536, 1.5, 0.8370536, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(215, 192, 215)","(0.8370536, 1.5, 0.8370536)","(256, 128, 256)"
1,IBSR_04,"(0.9375, 1.5, 0.9375, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(240, 192, 240)","(0.9375, 1.5, 0.9375)","(256, 128, 256)"
2,IBSR_08,"(1.0, 1.5, 1.0, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(256, 192, 256)","(1.0, 1.5, 1.0)","(256, 128, 256)"
3,IBSR_09,"(1.0, 1.5, 1.0, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(256, 192, 256)","(1.0, 1.5, 1.0)","(256, 128, 256)"
4,IBSR_07,"(1.0, 1.5, 1.0, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(256, 192, 256)","(1.0, 1.5, 1.0)","(256, 128, 256)"
5,IBSR_06,"(0.9375, 1.5, 0.9375, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(240, 192, 240)","(0.9375, 1.5, 0.9375)","(256, 128, 256)"
6,IBSR_03,"(0.9375, 1.5, 0.9375, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(240, 192, 240)","(0.9375, 1.5, 0.9375)","(256, 128, 256)"
7,IBSR_05,"(0.9375, 1.5, 0.9375, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(240, 192, 240)","(0.9375, 1.5, 0.9375)","(256, 128, 256)"
8,IBSR_16,"(0.8370536, 1.5, 0.8370536, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(215, 192, 215)","(0.8370536, 1.5, 0.8370536)","(256, 128, 256)"
9,IBSR_01,"(0.9375, 1.5, 0.9375, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(240, 192, 240)","(0.9375, 1.5, 0.9375)","(256, 128, 256)"


In [3]:
results_resampling = []
for val_set in pd_test_path.iterdir():
    case_results = {}
    if val_set.is_dir():
        # get original spacing and shape
        case_results['case'] = val_set.name
        o_img = nib.load(str(test_path/f'{val_set.name}/{val_set.name}.nii.gz'))
        case_results['o_spacing'] = o_img.header.get_zooms()
        case_results['o_shape'] = o_img.header.get_data_shape()
        
        # read and resample back the segmentation
        seg_img = nib.load(str(val_set/f'{val_set.name}_sseg.nii.gz'))
        case_results['pd_spacing'] = seg_img.header.get_zooms()
        case_results['pd_shape'] = seg_img.header.get_data_shape()
        seg_img_resampled = conform(seg_img, out_shape=case_results['o_shape'][:-1], 
                                                    voxel_size=case_results['o_spacing'][:-1], order=0)
        nib.save(seg_img_resampled, str(val_set/f'{val_set.name}_seg_resampled.nii.gz'))

        # read and resample back the resampled image
        img_rs = nib.load(str(val_set/f'{val_set.name}_resmp.nii.gz'))
        img_rs_back = conform(img_rs, out_shape=case_results['o_shape'][:-1], 
                                                    voxel_size=case_results['o_spacing'][:-1], order=4)
        nib.save(img_rs_back, str(val_set/f'{val_set.name}_resmp_back.nii.gz'))  

        case_results['new_spacing'] = seg_img_resampled.header.get_zooms()
        case_results['new_shape'] = seg_img_resampled.header.get_data_shape()
        results_resampling.append(case_results)
pd.DataFrame(results_resampling)

,case,o_spacing,o_shape,pd_spacing,pd_shape,new_spacing,new_shape
0,IBSR_10,"(1.0, 1.5, 1.0, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(256, 192, 256)","(1.0, 1.5, 1.0)","(256, 128, 256)"
1,IBSR_15,"(0.8370536, 1.5, 0.8370536, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(215, 192, 215)","(0.8370536, 1.5, 0.8370536)","(256, 128, 256)"
2,IBSR_02,"(0.9375, 1.5, 0.9375, 0.0)","(256, 128, 256, 1)","(1.0, 1.0, 1.0)","(240, 192, 240)","(0.9375, 1.5, 0.9375)","(256, 128, 256)"
